# NBA player points prediction

In [34]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt

## Get data with web scraping

In [35]:
def get_player_stats(year):
    # URL page we will scraping (see image above)
    url = "https://www.basketball-reference.com/leagues/NBA_{}_totals.html".format(year)
    # this is the HTML from the given URL
    html = urlopen(url)
    soup = BeautifulSoup(html, features="html.parser")

    # use findALL() to get the column headers
    soup.findAll('tr', limit=2)
    # use getText()to extract the text we need into a list
    headers = [th.getText() for th in 
        soup.findAll('tr', limit=2)[0].findAll('th')]
    # exclude the first column as we will not need the 
    # ranking order from Basketball Reference for the analysis
    headers = headers[1:]

    # avoid the first header row
    rows = soup.findAll('tr')[1:]

    player_stats = [[td.getText() for td 
        in rows[i].findAll('td')]
        for i in range(len(rows))]

    # Create DataFrame
    return pd.DataFrame(player_stats, columns=headers)

* Pos: Position.
* Age: Player age.
* G: Games.
* GS: Games started
* MP: Minutes played.
* FG (3P + 2P): Field goals.
* FGA (3PA + 2PA): Field goal attempts.
* FG%: Field goal percentage.
* 3P: 3 points field goals.
* 3PA: 3 point field goal attempts.
* 3P%: FG% on 3 points FGA’s (field goal attempts).
* 2P: 2 points field goals.
* 2PA: 2 point field goal attempts.
* 2P%: FG% on 2 points FGA’s (field goal attempts).
* FT:Free Throwns.
* FTA:Free Thrown Attempts.
* FT%: Free Throw Percentage.
* ORB: Offensive Rebounds.
* DRB: Defensive Rebounds.
* TRB: Total Rebounds
* AST: Assists.
* STL: Steals.
* BLK: Blocks.
* TOV:Turnovers.
* PF: Personal Fouls.
* PTS: Points.

In [36]:
original_data2019 = get_player_stats(2019)

In [37]:
original_data2020 = get_player_stats(2020)

In [38]:
data2019 = original_data2019.copy()
data2019

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Álex Abrines,SG,25,OKC,31,2,588,56,157,.357,...,.923,5,43,48,20,17,6,14,53,165
1,Quincy Acy,PF,28,PHO,10,0,123,4,18,.222,...,.700,3,22,25,8,1,4,4,24,17
2,Jaylen Adams,PG,22,ATL,34,1,428,38,110,.345,...,.778,11,49,60,65,14,5,28,45,108
3,Steven Adams,C,25,OKC,80,80,2669,481,809,.595,...,.500,391,369,760,124,117,76,135,204,1108
4,Bam Adebayo,C,21,MIA,82,28,1913,280,486,.576,...,.735,165,432,597,184,71,65,121,203,729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,Tyler Zeller,C,29,MEM,4,1,82,16,28,.571,...,.778,9,9,18,3,1,3,4,16,46
730,Ante Žižić,C,22,CLE,59,25,1082,183,331,.553,...,.705,108,212,320,53,13,22,61,113,459
731,Ivica Zubac,C,21,TOT,59,37,1040,212,379,.559,...,.802,115,247,362,63,14,51,70,137,525
732,Ivica Zubac,C,21,LAL,33,12,516,112,193,.580,...,.864,54,108,162,25,4,27,33,73,281


In [39]:
data2020 = original_data2020.copy()
data2020

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Steven Adams,C,26,OKC,63,63,1680,283,478,.592,...,.582,207,376,583,146,51,67,94,122,684
1,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,.557,...,.691,176,559,735,368,82,93,204,182,1146
2,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,.493,...,.827,103,289,392,129,36,87,74,128,1001
3,Kyle Alexander,C,23,MIA,2,0,13,1,2,.500,...,,2,1,3,0,0,0,1,1,2
4,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,266,.368,...,.676,9,75,84,89,17,8,54,57,267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,Trae Young,PG,21,ATL,60,60,2120,546,1249,.437,...,.860,32,223,255,560,65,8,289,104,1778
673,Cody Zeller,C,27,CHO,58,39,1341,251,479,.524,...,.682,160,251,411,88,40,25,75,140,642
674,Tyler Zeller,C,30,SAS,2,0,4,1,4,.250,...,,3,1,4,0,0,0,0,0,2
675,Ante Žižić,C,23,CLE,22,0,221,41,72,.569,...,.737,18,48,66,6,7,5,10,27,96


## Data Wrangling

In [40]:
def crossjoin(df1, df2):
    for index, row in df1.iterrows():
        player = df2.loc[df2['Player'] == row['Player']]
        if(not player.empty):
            continue
        else:
            df1.drop(index, inplace=True)
            df2.drop(player.index, inplace=True)
            
    return (df1, df2)

In [41]:
data2020, data2019 = crossjoin(data2020, data2019)
data2019, data2020 = crossjoin(data2019, data2020)

In [42]:
data2019.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [43]:
data2019 = data2019[['Player', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',',''), errors='ignore'))
data2020 = data2020[['Player','G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',',''), errors='ignore'))

### Create new column with total field goals attempts

In [44]:
data2019['TA'] = data2019['FGA'] + data2019['FTA'] 
data2020['TA'] = data2020['FGA'] + data2020['FTA'] 

## Correlation

In [45]:
corr2019 = data2019.corr()
corr2019.style.format(precision=3).background_gradient(cmap='coolwarm')

,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,TA
G,1.000,0.611,0.887,0.747,0.747,0.298,0.594,0.611,0.122,0.677,0.684,0.273,0.359,0.583,0.597,0.184,0.542,0.707,0.688,0.592,0.721,0.518,0.674,0.859,0.735,0.728
GS,0.611,1.000,0.839,0.799,0.794,0.218,0.596,0.604,0.141,0.740,0.753,0.157,0.240,0.682,0.686,0.200,0.514,0.721,0.690,0.659,0.732,0.516,0.748,0.724,0.793,0.786
MP,0.887,0.839,1.000,0.911,0.919,0.252,0.729,0.748,0.181,0.824,0.843,0.213,0.316,0.760,0.767,0.244,0.560,0.803,0.765,0.750,0.849,0.535,0.838,0.884,0.907,0.903
FG,0.747,0.799,0.911,1.000,0.985,0.300,0.687,0.703,0.138,0.951,0.962,0.218,0.297,0.891,0.894,0.235,0.586,0.811,0.779,0.765,0.772,0.536,0.906,0.785,0.993,0.986
FGA,0.747,0.794,0.919,0.985,1.000,0.194,0.771,0.793,0.176,0.897,0.930,0.143,0.228,0.879,0.870,0.274,0.480,0.753,0.704,0.789,0.788,0.451,0.905,0.760,0.989,0.992
FG%,0.298,0.218,0.252,0.300,0.194,1.000,-0.090,-0.109,0.041,0.410,0.338,0.794,0.882,0.253,0.296,-0.073,0.544,0.420,0.475,0.111,0.165,0.456,0.234,0.367,0.258,0.225
3P,0.594,0.596,0.729,0.687,0.771,-0.090,1.000,0.991,0.383,0.430,0.488,-0.014,0.163,0.575,0.519,0.374,0.030,0.392,0.301,0.589,0.609,0.107,0.612,0.519,0.740,0.726
3PA,0.611,0.604,0.748,0.703,0.793,-0.109,0.991,1.000,0.349,0.454,0.513,-0.008,0.131,0.600,0.548,0.362,0.041,0.411,0.319,0.620,0.639,0.124,0.644,0.538,0.756,0.750
3P%,0.122,0.141,0.181,0.138,0.176,0.041,0.383,0.349,1.000,0.010,0.040,-0.111,0.374,0.079,0.038,0.317,-0.172,0.022,-0.033,0.121,0.117,-0.098,0.080,0.084,0.163,0.145
2P,0.677,0.740,0.824,0.951,0.897,0.410,0.430,0.454,0.010,1.000,0.989,0.276,0.300,0.864,0.891,0.133,0.716,0.842,0.840,0.700,0.702,0.621,0.866,0.756,0.920,0.918


In [46]:
corr2019[corr2019['PTS'] > 0.7]['PTS']

G      0.734652
GS     0.792749
MP     0.906744
FG     0.992596
FGA    0.989421
3P     0.739717
3PA    0.755715
2P     0.919859
2PA    0.937531
FT     0.921784
FTA    0.914765
DRB    0.786505
TRB    0.744496
AST    0.774326
STL    0.772120
TOV    0.909945
PF     0.767228
PTS    1.000000
TA     0.995355
Name: PTS, dtype: float64

### Most correlated vars

In [47]:
corr_columns = list(corr2019[corr2019['PTS'] > 0.7]['PTS'].keys())
corr_columns.append('Player')
data2019 = data2019[corr_columns]

In [48]:
data2019

,G,GS,MP,FG,FGA,3P,3PA,2P,2PA,FT,FTA,DRB,TRB,AST,STL,TOV,PF,PTS,TA,Player
3,80,80,2669,481,809,0,2,481,807,146,292,369,760,124,117,135,204,1108,1101,Steven Adams
4,82,28,1913,280,486,3,15,277,471,166,226,432,597,184,71,121,203,729,712,Bam Adebayo
7,81,81,2687,684,1319,10,42,674,1277,349,412,493,744,194,43,144,179,1727,1731,LaMarcus Aldridge
9,38,2,416,67,178,32,99,35,79,45,60,20,23,25,6,33,47,211,238,Grayson Allen
10,80,80,2096,335,568,6,45,329,523,197,278,481,672,110,43,103,184,873,846,Jarrett Allen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,4,1,82,16,28,0,0,16,28,14,18,9,18,3,1,4,16,46,46,Tyler Zeller
730,59,25,1082,183,331,0,0,183,331,93,132,212,320,53,13,61,113,459,463,Ante Žižić
731,59,37,1040,212,379,0,0,212,379,101,126,247,362,63,14,70,137,525,505,Ivica Zubac
732,33,12,516,112,193,0,0,112,193,57,66,108,162,25,4,33,73,281,259,Ivica Zubac


In [49]:
corr2019 = data2019.corr()
corr2019.style.format(precision=3).background_gradient(cmap='coolwarm')

,G,GS,MP,FG,FGA,3P,3PA,2P,2PA,FT,FTA,DRB,TRB,AST,STL,TOV,PF,PTS,TA
G,1.000,0.611,0.887,0.747,0.747,0.594,0.611,0.677,0.684,0.583,0.597,0.707,0.688,0.592,0.721,0.674,0.859,0.735,0.728
GS,0.611,1.000,0.839,0.799,0.794,0.596,0.604,0.740,0.753,0.682,0.686,0.721,0.690,0.659,0.732,0.748,0.724,0.793,0.786
MP,0.887,0.839,1.000,0.911,0.919,0.729,0.748,0.824,0.843,0.760,0.767,0.803,0.765,0.750,0.849,0.838,0.884,0.907,0.903
FG,0.747,0.799,0.911,1.000,0.985,0.687,0.703,0.951,0.962,0.891,0.894,0.811,0.779,0.765,0.772,0.906,0.785,0.993,0.986
FGA,0.747,0.794,0.919,0.985,1.000,0.771,0.793,0.897,0.930,0.879,0.870,0.753,0.704,0.789,0.788,0.905,0.760,0.989,0.992
3P,0.594,0.596,0.729,0.687,0.771,1.000,0.991,0.430,0.488,0.575,0.519,0.392,0.301,0.589,0.609,0.612,0.519,0.740,0.726
3PA,0.611,0.604,0.748,0.703,0.793,0.991,1.000,0.454,0.513,0.600,0.548,0.411,0.319,0.620,0.639,0.644,0.538,0.756,0.750
2P,0.677,0.740,0.824,0.951,0.897,0.430,0.454,1.000,0.989,0.864,0.891,0.842,0.840,0.700,0.702,0.866,0.756,0.920,0.918
2PA,0.684,0.753,0.843,0.962,0.930,0.488,0.513,0.989,1.000,0.876,0.894,0.813,0.799,0.737,0.725,0.886,0.746,0.938,0.944
FT,0.583,0.682,0.760,0.891,0.879,0.575,0.600,0.864,0.876,1.000,0.988,0.732,0.696,0.718,0.668,0.868,0.652,0.922,0.929


## Build model

### Sum data of a player
If player was in several teams in the season we maybe should sum his data in one row, for represent the entire season in one row per player.

In [50]:
data2019 = data2019.groupby('Player', as_index=False)[corr_columns].sum()

In [51]:
data2019

,Player,G,GS,MP,FG,FGA,3P,3PA,2P,2PA,FT,FTA,DRB,TRB,AST,STL,TOV,PF,PTS,TA
0,Aaron Gordon,78,78,2633,470,1046,121,347,349,699,185,253,445,574,289,57,162,172,1246,1299
1,Aaron Holiday,50,0,646,105,262,43,127,62,135,41,50,62,67,87,21,40,71,294,312
2,Abdel Nader,61,1,694,91,215,32,100,59,115,27,36,102,116,20,20,26,68,241,251
3,Al Horford,68,68,1973,387,723,73,203,314,520,78,95,338,458,283,59,102,126,925,818
4,Al-Farouq Aminu,81,81,2292,257,593,96,280,161,313,150,173,498,610,104,68,72,143,760,766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,Yogi Ferrell,71,3,1067,153,352,54,149,99,203,60,67,96,109,137,36,40,64,420,419
396,Yuta Watanabe,15,0,174,15,51,2,16,13,35,7,10,27,31,8,4,6,11,39,61
397,Zach Collins,77,0,1356,189,400,40,121,149,279,94,126,215,324,71,25,77,174,512,526
398,Zach LaVine,63,62,2171,530,1135,120,321,410,814,312,375,254,294,283,60,215,140,1492,1510


In [52]:
data2020 = data2020.groupby('Player', as_index=False)[corr_columns].sum()

### Model with total attempts

In [53]:
lm_TA = smf.ols(formula='PTS~TA', data=data2019).fit()
lm_TA.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    PTS   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 4.056e+04
Date:                Mon, 27 Dec 2021   Prob (F-statistic):               0.00
Time:                        16:52:54   Log-Likelihood:                -2175.6
No. Observations:                 400   AIC:                             4355.
Df Residuals:                     398   BIC:                             4363.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.0587      4.508     -0.457      0.648     -10.922       6.805
TA             0.9948      0.005    201.388      0.000       0.985       1.004
==============================================================================
Omnibus:                       32.340   Durbin-Watson:                   1.876
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              118.754
Skew:                           0.209   Prob(JB):                     1.63e-26
Kurtosis:                       5.636   Cond. No.                     1.47e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.47e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [54]:
data2020['PTS_pred_TA'] = lm_TA.params[0] + lm_TA.params[1] * data2019['TA']

### Model with minutes played

In [55]:
lm_MP = smf.ols(formula='PTS~MP', data=data2019).fit()
lm_MP.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    PTS   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     1618.
Date:                Mon, 27 Dec 2021   Prob (F-statistic):          2.71e-142
Time:                        16:52:54   Log-Likelihood:                -2777.9
No. Observations:                 400   AIC:                             5560.
Df Residuals:                     398   BIC:                             5568.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -58.2706     22.890     -2.546      0.011    -103.271     -13.270
MP             0.4954      0.012     40.224      0.000       0.471       0.520
==============================================================================
Omnibus:                       91.333   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              355.914
Skew:                           0.954   Prob(JB):                     5.18e-78
Kurtosis:                       7.209   Cond. No.                     3.38e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.38e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [56]:
data2020['PTS_pred_MP'] = lm_MP.params[0] + lm_MP.params[1] * data2019['MP']

### Model with assits

In [57]:
lm_AST = smf.ols(formula='PTS~AST', data=data2019).fit()
lm_AST.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    PTS   R-squared:                       0.552
Model:                            OLS   Adj. R-squared:                  0.551
Method:                 Least Squares   F-statistic:                     491.0
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           1.90e-71
Time:                        16:52:54   Log-Likelihood:                -2941.6
No. Observations:                 400   AIC:                             5887.
Df Residuals:                     398   BIC:                             5895.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    264.8252     27.646      9.579      0.000     210.475     319.175
AST            2.9097      0.131     22.158      0.000       2.652       3.168
==============================================================================
Omnibus:                       49.213   Durbin-Watson:                   2.117
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.257
Skew:                           0.714   Prob(JB):                     1.53e-20
Kurtosis:                       4.853   Cond. No.                         307.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [58]:
data2020['PTS_pred_AST'] = lm_AST.params[0] + lm_AST.params[1] * data2019['AST']

### Model with steals

In [59]:
lm_STL = smf.ols(formula='PTS~STL', data=data2019).fit()
lm_STL.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    PTS   R-squared:                       0.565
Model:                            OLS   Adj. R-squared:                  0.564
Method:                 Least Squares   F-statistic:                     517.5
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           5.40e-74
Time:                        16:52:54   Log-Likelihood:                -2935.7
No. Observations:                 400   AIC:                             5875.
Df Residuals:                     398   BIC:                             5883.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    179.3376     29.909      5.996      0.000     120.537     238.138
STL           10.7798      0.474     22.748      0.000       9.848      11.711
==============================================================================
Omnibus:                       78.102   Durbin-Watson:                   2.087
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              184.943
Skew:                           0.978   Prob(JB):                     6.92e-41
Kurtosis:                       5.696   Cond. No.                         101.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
data2020['PTS_pred_STL'] = lm_STL.params[0] + lm_STL.params[1] * data2019['STL']

In [61]:
data2020

,Player,G,GS,MP,FG,FGA,3P,3PA,2P,2PA,...,AST,STL,TOV,PF,PTS,TA,PTS_pred_TA,PTS_pred_MP,PTS_pred_AST,PTS_pred_STL
0,Aaron Gordon,62,62,2017,335,767,73,237,262,530,...,228,51,100,125,894,991,1290.145920,1246.074479,1105.738512,793.784825
1,Aaron Holiday,66,33,1617,233,563,87,221,146,342,...,225,55,88,120,627,650,308.309127,261.747183,517.972109,405.712891
2,Abdel Nader,55,6,867,123,263,48,128,75,135,...,38,23,43,78,345,329,247.628231,285.525597,323.019887,394.933115
3,Al Horford,67,61,2025,319,709,99,283,220,426,...,270,52,80,142,798,789,811.662134,919.121275,1088.280104,815.344377
4,Al-Farouq Aminu,18,2,380,25,86,9,36,16,50,...,21,18,17,27,78,115,759.934157,1077.148657,567.437599,912.362361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,Yogi Ferrell,50,0,530,79,188,24,79,55,109,...,69,18,28,38,218,230,414.749387,470.303696,663.458843,567.409531
396,Yuta Watanabe,18,0,105,15,34,3,8,12,26,...,5,5,2,9,36,42,58.622161,27.926104,288.103071,222.456700
397,Zach Collins,11,11,290,32,68,7,19,25,49,...,16,5,14,36,77,76,521.189648,613.469568,471.416355,448.831995
398,Zach LaVine,60,60,2085,539,1199,184,484,355,715,...,254,88,206,131,1530,1533,1500.042134,1017.207236,1088.280104,826.124153


In [62]:
def percentage_change(col1,col2):
    return abs(((col2 - col1) / col1) * 100)

In [63]:
data2020['change_pred_TA'] = percentage_change(data2020['PTS'], data2020['PTS_pred_TA'])
data2020['change_pred_MP'] = percentage_change(data2020['PTS'], data2020['PTS_pred_MP'])
data2020['change_pred_AST'] = percentage_change(data2020['PTS'], data2020['PTS_pred_AST'])
data2020['change_pred_STL'] = percentage_change(data2020['PTS'], data2020['PTS_pred_STL'])

In [64]:
data2020.columns

Index(['Player', 'G', 'GS', 'MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT',
       'FTA', 'DRB', 'TRB', 'AST', 'STL', 'TOV', 'PF', 'PTS', 'TA',
       'PTS_pred_TA', 'PTS_pred_MP', 'PTS_pred_AST', 'PTS_pred_STL',
       'change_pred_TA', 'change_pred_MP', 'change_pred_AST',
       'change_pred_STL'],
      dtype='object')

In [65]:
def min_name_dict(test_dict):
    min_value = min(test_dict.values())
    return [key for key in test_dict if test_dict[key] == min_value][0]

data2020['best'] = data2020.apply(lambda row: min_name_dict(row[['change_pred_TA', 'change_pred_MP', 'change_pred_AST',
       'change_pred_STL']].to_dict()).split('_')[-1], axis=1)

In [66]:
data2020['best'].value_counts()

TA     132
AST    121
MP      74
STL     73
Name: best, dtype: int64